# Session 1 - Getting Started

This session aims to build a simple [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with one step to preprocess the dataset.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Let's make sure we are running the latest version of the SakeMaker's SDK. 
# Restart the notebook after you upgrade the library.

!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker
!pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.22 requires botocore==1.23.22, but you have botocore 1.29.110 which is incompatible.
awscli 1.22.22 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.6.0 which is incompatible.
Name: sagemaker
Version: 2.145.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, protobuf3-to-dict, PyYAML, schema, smdebug-rulesconfig
Required-by: 


In [19]:
import os
import sagemaker
import numpy as np
import boto3
import json
import pandas as pd
import numpy as np
import urllib.request
import argparse
import tempfile

from botocore.exceptions import ClientError
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig


role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Step 1 - Creating an S3 Bucket

We need to create an S3 bucket where we will upload everything we need during the program.

Make sure you set `BUCKET` to the name of the bucket you want to use.

In [5]:
BUCKET = "mlschool"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool"
}


## Step 2 - Downloading the Dataset

We can now download the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and store it in S3.

In [6]:
S3_FILEPATH = f"s3://{BUCKET}/penguins"
DATA_FILEPATH = "penguins/data.csv"

# Download the official Penguins dataset and store it locally.
urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

# Upload the dataset to S3. We need to do this to make it available to 
# the preprocessing step.
INPUT_DATA_URI = sagemaker.s3.S3Uploader.upload(
    local_path=DATA_FILEPATH, 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Dataset S3 location: {INPUT_DATA_URI}")

Dataset S3 location: s3://mlschool/penguins/data.csv


We can now load and display the dataset.

In [7]:
df = pd.read_csv(DATA_FILEPATH)
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


## Step 3 - Preprocessing the Dataset

Let's create a script to do feature engineering on the original dataset. This script should also split the data into train, validation, and a test set.

In [8]:
%%writefile penguins/preprocessor.py

import os
import numpy as np
import pandas as pd
import tempfile

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler


BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIR) / "input" / "data.csv"


def save_splits(base_dir, train, validation, test):
    """
    Saves the supplied datasets to disk.
    """
    
    train_path = Path(base_dir) / "train" 
    validation_path = Path(base_dir) / "validation" 
    test_path = Path(base_dir) / "test"
    
    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def preprocess(base_dir, data_filepath):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """
    
    df = pd.read_csv(data_filepath)
    
    numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]

    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
            ("categorical", categorical_preprocessor, ["island"]),
        ]
    )
    

    X = df.drop(["sex"], axis=1)
    columns = list(X.columns)
    
    X = X.to_numpy()
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])
    
    X_train = pd.DataFrame(train, columns=columns)
    X_validation = pd.DataFrame(validation, columns=columns)
    X_test = pd.DataFrame(test, columns=columns)
    
    y_train = X_train.species
    y_validation = X_validation.species
    y_test = X_test.species

    X_train.drop(["species"], axis=1, inplace=True)
    X_validation.drop(["species"], axis=1, inplace=True)
    X_test.drop(["species"], axis=1, inplace=True)
    
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    label_encoder = LabelEncoder()

    y_train = label_encoder.fit_transform(y_train)
    y_validation = label_encoder.transform(y_validation)
    y_test = label_encoder.transform(y_test)
    
    
    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)
    
    save_splits(base_dir, train, validation, test)
    
    
if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH)


Overwriting penguins/preprocessor.py


We can now load the script we just created and run it locally to ensure it creates the 3 splits. 

Having a way to run scripts locally is crucial to shorten the development feedback.

In [9]:
from penguins.preprocessor import preprocess

with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    print(f"Splits: {os.listdir(directory)}")
    print(f"Train: {os.listdir(Path(directory) / 'train')}")
    print(f"Validation: {os.listdir(Path(directory) / 'validation')}")
    print(f"Test: {os.listdir(Path(directory) / 'test')}")

Splits: ['train', 'validation', 'test']
Train: ['train.csv']
Validation: ['validation.csv']
Test: ['test.csv']


## Step 4 - Pipeline Configuration

When we create a SageMaker Pipeline we can specify a list of paramaters that we can use throughout the individual pipeline steps. To read more about these parameters, check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html).

These are the parameters that will use in our pipeline:

* `dataset_location`: This parameter represents the location of the dataset in S3. We will use this parameter during the preprocessing step to access the dataset.
* `preprocessor_destination`: We need to define the location where the preprocessing step will be storing the dataset splits to avoid SageMaker from appending a timestamp to their auto-generated location. If we let SageMaker use a timestamp, we can't cache this step.

In [10]:
dataset_location = ParameterString(
    name="dataset_location",
    default_value=INPUT_DATA_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f'{S3_FILEPATH}/preprocessing',
)

## Step 5 - Caching Pipeline Steps

While you are building your pipeline, you don't want to rerun every step of the process unless you expect a different result. Instead, you can instruct SageMaker to reuse the result of a previous successful run of a pipeline step.

You can accomplish this by caching your steps. You can find more information about this topic in [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html).

Getting caching to work is tricky, and you will find SageMaker missing the cache frequently. Whenever that happens, you need to dig and figure out how to adjust the step configuration to prevent SageMaker from autogenerating data that prevents a cache hit. For example, to cache the preprocessing step we need to define the destination of the processing job to prevent SageMaker from using an autogenerated timestamp.

In [11]:
# We'll use this cache configuration to cache individual steps for 
# a maximum of 5 days.
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="5d"
)

## Step 6 - Setting up a Processing Step

The first step we need in our pipeline is a Processing Step to run the preprocessing script. Check the [Processing Step documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) for more information. To run our script, we need access to Scikit-Learn, so we can use the [SKLearnProcessor]((https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) processor that comes out-of-the-box with the SageMaker's Python SDK.

The input of this step will be the dataset location, and the output will be the location of the three sets.

In [12]:
sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

preprocess_step = ProcessingStep(
    name="penguins-preprocessing-step",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
    ],
    code="penguins/preprocessor.py",
    cache_config=cache_config
)

## Step 7 - Defining and Running the Pipeline

We can now define and run the SageMaker Pipeline. Check [Pipeline Structure and Execution](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-pipeline.html) for more information about how to define a pipeline.


In [14]:

session1_pipeline = Pipeline(
    name="session1-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
    ]
)

In [15]:
session1_pipeline.upsert(role_arn=role)
execution = session1_pipeline.start()

## Step 8 - Cleaning up

Before you finish, don't forget to clean up after you.

In [21]:
pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session1-penguins-pipeline',
 'ResponseMetadata': {'RequestId': '3d9c3497-4a2b-44fa-afdb-0ce1bf7377e4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3d9c3497-4a2b-44fa-afdb-0ce1bf7377e4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Mon, 10 Apr 2023 21:36:49 GMT'},
  'RetryAttempts': 0}}

## Assignments

1. Set up an Amazon SageMaker domain using the Standard Setup. Make sure you set the network configuration to VPC Only. Create a new execution role and ensure it has access to the S3 bucket you’ll use during this class. You can also specify “Any S3 bucket” if you want this role to access every S3 bucket in your AWS account.

2. Create a GitHub repository and clone it from inside SageMaker Studio. We’ll use this repository to store the code used during this program.

3. Configure your SageMaker Studio session to store your name and email address and cache your credentials. You can use the following commands from a Terminal window:

```bash
$ git config --global user.name "John Doe"
$ git config --global user.email johndoe@example.com
$ git config --global credential.helper store
```

4. Prepare the MNIST dataset. Throughout the course, you will create a SageMaker pipeline to work with the MNIST dataset. MNIST is popular and relatively small, so it's easy to find pre-packaged versions of it. We aren't going to use those. Instead, we will simulate a practical scenario where the data is stored in the filesystem. To accomplish this, you will load a pre-packaged version of MNIST, save it to disk, and upload it to an S3 bucket. Complete the section "Prepare the MNIST Dataset" below.

5. Setup a SageMaker Pipeline with a preprocessing step where you split the MNIST dataset into a train and a test set.

### Prepare the MNIST Dataset

These are the steps you need to follow to prepare the data:

1. Create the S3 bucket to upload the dataset.
2. Load the MNIST dataset from the Keras built-in collection of small datasets, convert it into images, and save them to the disk.
3. Upload the dataset to the S3 bucket.


#### Create the dataset

We want to load the Keras' built-in version of MNIST and save it to disk so we can later upload it to S3.

There are 70,000 images. Running this cell will take some time, so this is the perfect moment to walk around and grab some coffee. Fortunately, we only need to do this once.

In [ ]:
import numpy as np
import tensorflow as tf

from PIL import Image
from pathlib import Path
from tensorflow.keras.datasets import mnist


def save(dataset, split, images, labels):
    """
    This function saves the handwritten digits to disk as PNG files.
    
    Every image will be saved inside a folder corresponding to 
    its label. For example, a digit from the train set representing 
    the number 3 will be saved inside the folder `~/train/3`.
    """
    
    for index, (image, label) in enumerate(zip(images, labels)):
        im = Image.fromarray(image)

        path = dataset / split / str(label)
        path.mkdir(parents=True, exist_ok=True)
        
        im.save(path / f"{index}.png")
        

# We will save the dataset in the home directory, inside a folder
# named `dataset`.
dataset = Path.home() / "dataset" 

# We want to make sure we don't generate the images if the dataset
# already exists.
if not dataset.exists():
    # Load the MNIST dataset using the Keras library. This returns the
    # dataset in numpy arrays.
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Use the function we created to save the data to disk.
    save(dataset, split="train", images=X_train, labels=y_train)
    save(dataset, split="test", images=X_test, labels=y_test)

#### Upload the data to S3

Now that we exported the MNIST dataset to the filesystem, we need to upload them to an S3 bucket. The easiest way to do this is to use the AWS CLI.

This command will also take a while to finish.

In [ ]:
!aws s3 cp $dataset s3://$BUCKET/mnist --recursive

## Additional Notes

1. Amazon SageMaker is free to try. Your free tier starts from the first month when you create your first SageMaker resource and lasts 2 months. Check out the [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) for more information.

2. We’ll be working extensively with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) and [SageMaker’s Python SDK](https://sagemaker.readthedocs.io/en/stable/).

3. This notebook uses a Scikit-Learn Pipeline to transform the dataset. You should always orchestrate your transformations using pipelines. Check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for more details.

4. The preprocessing script uses `np.split()` to split the dataset into 3 different splits. It's a neat way of getting the three splits with a single instruction.

5. Keras offers a [list of built-in vectorized datasets](https://www.notion.so/Bnomial-RESTful-API-4ecf85043b484ec994d7f70c56abfe27) in NumPy format. You can load any of these datasets with a single line of code, making them convenient.

6. Converting a NumPy array into an image you can save and visualize is a useful trick to know. Check the `Image.fromarray()` function from the `PIL` library.

7. The [command line interface](https://docs.aws.amazon.com/cli/latest/index.html) is a simple way to interact with the AWS services. You can combine Python code with bash commands in the same notebook cell, which makes notebooks a very flexible tool.

8. Check Python’s `pathlib` module. Since Python 3.4, this module offers a clean way to interact with the filesystem.

9. This notebook uses the `%%writefile` cell magic. There's a whole list of [line and cell magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) you can start using in your code.

# Session 2 - Training

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep

## Step 1 - Training the Model

This script is responsible from training a simple neural network on the train data, validating the model, and saving it so we can later use it. Training

In [24]:
%%writefile penguins/train.py

import os
import argparse

import numpy as np
import pandas as pd
import random
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dropout


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting penguins/train.py


In [25]:
from penguins.preprocessor import preprocess
from penguins.train import train

with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )

Epoch 1/50
8/8 - 1s - loss: 1.2794 - accuracy: 0.2636 - val_loss: 1.1317 - val_accuracy: 0.3725
Epoch 2/50
8/8 - 0s - loss: 1.1494 - accuracy: 0.3054 - val_loss: 1.0345 - val_accuracy: 0.4510
Epoch 3/50
8/8 - 0s - loss: 1.0582 - accuracy: 0.3473 - val_loss: 0.9637 - val_accuracy: 0.4510
Epoch 4/50
8/8 - 0s - loss: 0.9883 - accuracy: 0.4226 - val_loss: 0.9054 - val_accuracy: 0.5098
Epoch 5/50
8/8 - 0s - loss: 0.9313 - accuracy: 0.4770 - val_loss: 0.8580 - val_accuracy: 0.5098
Epoch 6/50
8/8 - 0s - loss: 0.8835 - accuracy: 0.5314 - val_loss: 0.8156 - val_accuracy: 0.5490
Epoch 7/50
8/8 - 0s - loss: 0.8409 - accuracy: 0.5941 - val_loss: 0.7795 - val_accuracy: 0.6078
Epoch 8/50
8/8 - 0s - loss: 0.8041 - accuracy: 0.6527 - val_loss: 0.7470 - val_accuracy: 0.6275
Epoch 9/50
8/8 - 0s - loss: 0.7710 - accuracy: 0.7155 - val_loss: 0.7169 - val_accuracy: 0.6667
Epoch 10/50
8/8 - 0s - loss: 0.7404 - accuracy: 0.7908 - val_loss: 0.6891 - val_accuracy: 0.7843
Epoch 11/50
8/8 - 0s - loss: 0.7131 - a

INFO:tensorflow:Assets written to: /tmp/tmp4y2l190c/model/001/assets


## Step 2 - Setting up a Training Step

In [26]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32
}

estimator = TensorFlow(
    entry_point="penguins/train.py",
    role=role,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.large",
    instance_count=1,
    py_version="py37",
    framework_version="2.4",
    script_mode=True,
    disable_profiler=True
)

training_step = TrainingStep(
    name="penguins-training-step",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 3 - Defining and Running the Pipeline

In [27]:
session2_pipeline = Pipeline(
    name="session2-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        training_step
    ]
)

In [28]:
session2_pipeline.upsert(role_arn=role)
execution = session2_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Step 4 - Cleaning up

Before you finish, don't forget to clean up after you.

In [56]:
session2_pipeline.delete()

ResourceNotFound: An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session2-penguins-pipeline' does not exist.

## Assignments

1. Modify the training script so it accepts the `learning_rate` as a new hyperparameter using the list of hyperparameters supplied to the Estimator.

2. Replace the TensorFlow Estimator with a Pytorch Estimator. Check [this page](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#create-an-estimator) for an example of how to create a PyTorch Estimator. You'll need to create a new training script that builds a PyTorch model to solve the problem.

3. Modify the SageMaker Pipeline you created for the MNIST project and add a training step. That step should only receive one channel with the train data. 

## Additional Notes

# Session 3 - Tuning

In [45]:
from sagemaker.workflow.pipeline_context import PipelineSession

from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter

In [52]:
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 50)
}

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

    
tuning_step = TuningStep(
    name = "penguins-tuning-step",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

In [53]:
session3_pipeline = Pipeline(
    name="session3-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step
    ]
)

In [54]:
session3_pipeline.upsert(role_arn=role)
execution = session3_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [57]:
session3_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session3-penguins-pipeline',
 'ResponseMetadata': {'RequestId': '133a079b-6c85-429d-9927-ad9206f7e53a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '133a079b-6c85-429d-9927-ad9206f7e53a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Mon, 10 Apr 2023 22:41:56 GMT'},
  'RetryAttempts': 0}}